I recently came across an [article](https://www.nature.com/articles/s41524-018-0085-8) by Stanev *et al.* which used the [random forest algorithm](https://en.wikipedia.org/wiki/Random_forest) to predict things about the superconducting critical temperatures ($T_\mathrm{c}$) of various compounds. One particularly surprising result was that when trained on a large data set of about 12 000 structures, the algorithm could determine with respectable accuracy whether the $T_\mathrm{c}$ of a compound was less than 10 K based only on information of the atomic constituents. In this blog post, I use the `sklearn` Python library to reproduce the results, and find an accuracy of 92%, consistent with the results in the paper.

First, we look at attributes relating to each atom in the compound, such as their column in the periodic table. Additionally, quantities relating to the elemental forms of each atom, such as the melting temperature and band gap, are considered. Since each compound contains multiple atoms, the random forest receives statistical quantities such as the mean, standard deviation, and maximum value for each atom. (These quantities are generated by a software called [Magpie](https://bitbucket.org/wolverton/magpie)). We cannot simply pass in the attributes for each atom, because the random forest algorithm can only receive a fixed number of features, and not every compound has the same number of atoms.

The reason why only simple atomic attributes were considered was because there is very limited data on most superconducting materials. In particular, the [SUPERCON database](http://supercon.nims.go.jp/) from the Japanese [National Institute for Materials Science (NIMS)](https://www.nims.go.jp/) contains the chemical formulas and $T_\mathrm{c}$ values for many compounds, but contains nothing else.

The IPython notebook for this work, as well as the data required to train and test the model, are available on my [GitHub](https://github.com/caesiumong/supercon_random_forest).

Without further ado, let's get started!

 First, let's import the relevant classes and functions from `sklearn`. We'll also need `pandas` to process the `csv` files containing our data.

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score

import pandas as pd

Next, we need to load the data from the SUPERCON database. This data is in `csv` format as `supercon_nims_data.csv`. Additionally, we need to load the data from atomic attributes generated by Magpie. 

In [2]:
nims_data = pd.read_csv("./supercon_nims_data.csv")
mp_data = pd.read_csv("./supercon_magpie_data.csv")

Now, let's have a look at the NIMS data. As we can see, it contains no information except for $T_\mathrm{c}$ and chemical formula.

In [3]:
nims_data.head()

,name,Tc
0,Ba0.4K0.6Fe2As2,31.20
1,Ca0.4Ba1.25La1.25Cu3O6.98,40.10
2,Mo0.39Ru0.61,6.90
3,Tm4Os6Sn19,1.10
4,Nd1Bi0.99Pb0.01S2F0.3O0.7,4.85


We can also have a look at the data generated by Magpie.

In [4]:
mp_data.head()

,NComp,Comp_L2Norm,Comp_L3Norm,Comp_L5Norm,Comp_L7Norm,Comp_L10Norm,mean_Number,maxdiff_Number,dev_Number,max_Number,...,min_SpaceGroupNumber,most_SpaceGroupNumber,frac_sValence,frac_pValence,frac_dValence,frac_fValence,CanFormIonic,MaxIonicChar,MeanIonicChar,Class
0,4.0,0.583781,0.506891,0.459606,0.441643,0.428710,30.360000,37.0,6.214400,56.0,...,166.0,197.5,0.198312,0.126582,0.675105,0.000000,0.0,0.370230,0.101293,None
1,5.0,0.606413,0.557901,0.543545,0.542136,0.541937,22.677795,49.0,16.074864,57.0,...,12.0,12.0,0.277798,0.340779,0.381423,0.000000,0.0,0.803211,0.321647,None
2,2.0,0.724017,0.659084,0.622509,0.613736,0.610693,43.220000,2.0,0.951600,44.0,...,194.0,194.0,0.138504,0.000000,0.861496,0.000000,0.0,0.000400,0.000190,None
3,3.0,0.700772,0.663969,0.655637,0.655203,0.655173,58.000000,26.0,10.482759,76.0,...,141.0,141.0,0.126638,0.082969,0.493450,0.296943,0.0,0.201983,0.036805,None
4,6.0,0.512258,0.435720,0.405168,0.400893,0.400075,36.658000,75.0,27.869600,83.0,...,12.0,70.0,0.187652,0.286921,0.187652,0.337774,0.0,0.866866,0.228340,None


Now, let's join these two `DataFrame`s together and create a column which tells us whether a compound has a $T_\mathrm{c}$ above 10 K. This column will serve as our ground truth for our model.

In [5]:
df = nims_data.join(mp_data)
df["over_10"] = df["Tc"] > 10
df.head()

,name,Tc,NComp,Comp_L2Norm,Comp_L3Norm,Comp_L5Norm,Comp_L7Norm,Comp_L10Norm,mean_Number,maxdiff_Number,...,most_SpaceGroupNumber,frac_sValence,frac_pValence,frac_dValence,frac_fValence,CanFormIonic,MaxIonicChar,MeanIonicChar,Class,over_10
0,Ba0.4K0.6Fe2As2,31.20,4.0,0.583781,0.506891,0.459606,0.441643,0.428710,30.360000,37.0,...,197.5,0.198312,0.126582,0.675105,0.000000,0.0,0.370230,0.101293,None,True
1,Ca0.4Ba1.25La1.25Cu3O6.98,40.10,5.0,0.606413,0.557901,0.543545,0.542136,0.541937,22.677795,49.0,...,12.0,0.277798,0.340779,0.381423,0.000000,0.0,0.803211,0.321647,None,True
2,Mo0.39Ru0.61,6.90,2.0,0.724017,0.659084,0.622509,0.613736,0.610693,43.220000,2.0,...,194.0,0.138504,0.000000,0.861496,0.000000,0.0,0.000400,0.000190,None,False
3,Tm4Os6Sn19,1.10,3.0,0.700772,0.663969,0.655637,0.655203,0.655173,58.000000,26.0,...,141.0,0.126638,0.082969,0.493450,0.296943,0.0,0.201983,0.036805,None,False
4,Nd1Bi0.99Pb0.01S2F0.3O0.7,4.85,6.0,0.512258,0.435720,0.405168,0.400893,0.400075,36.658000,75.0,...,70.0,0.187652,0.286921,0.187652,0.337774,0.0,0.866866,0.228340,None,False


Great! Our data is almost ready to be fed into the random forest classifier. For our features, we can use pretty much all of the columns. We won't include the name (because it's a string and the algorithm doesn't know how to interpret it) and we won't include the $T_\mathrm{c}$ (because that's cheating). Also, for some reason, `magpie` seems to generate a column full of `None` values, so we'll delete that as well. 

For our ground truth (our `y`), we will simply take the column which indicates whether the $T_\mathrm{c}$ is above 10 K or not.

I will use `sklearn`'s `train_test_split` function to separate our data into training (`Xtr` and `ytr`) and test (`Xte` and `yte`) sets. This way, we can check if what our model learned on the training set can generalize to another set. (The `random_state=42` is simply for reproducibility purposes.)

In [6]:
features = df.columns[2:-2]

X = df[features]
y = df["over_10"]

Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.15, random_state=42)

print("Number of training examples: %i \t number of features: %i" %
      (Xtr.shape[0], Xtr.shape[1]))

print("Number of testing examples:  %i \t number of features: %i" %
      (Xte.shape[0], Xtr.shape[1]))

Number of training examples: 13951 	 number of features: 145
Number of testing examples:  2463 	 number of features: 145


Now, let's define our random forest classifier and fit it on our training data. The hyperparameters set below seem to work well, but one could conceivably use the `GridSearchCV` function, which tests different sets of hyperparameters to find which ones yield the best performance. Again, the `random_state=42` is for reproducibility.

In [7]:
clf = RandomForestClassifier(n_jobs=-1, random_state=42, n_estimators=2000,
                             max_depth=50, min_samples_leaf=4,
                             min_samples_split=4)
clf.fit(Xtr, ytr)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=50, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=4, min_samples_split=4,
            min_weight_fraction_leaf=0.0, n_estimators=2000, n_jobs=-1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

Now, let's see how our model performed on our test set. We can look at the accuracy, which simply calculates the ratio of the number of times the algorithm correctly predicted to the total number of examples in the test set. This was the metric used in the paper. Additionally, we can also look at the $F_1$ score, which adjusts for the fact that the number of examples in each class may be different.

In [8]:
yhat = clf.predict(Xte)

print("Accuracy score: %.3f" % accuracy_score(yte, yhat))
print("F1 score: %.3f" % f1_score(yte, yhat))

Accuracy score: 0.918
F1 score: 0.893


It looks like we got 91.8%, which reproduces the result of 92% in the paper. Hurray!